In [1]:
import pandas as pd
import numpy as np
from transformers import Trainer, AutoModelForSequenceClassification
from transformers import AutoTokenizer
    
from datasets import load_dataset

In [3]:
model = AutoModelForSequenceClassification.from_pretrained('../0_models/bert-rand-1m-3ep-rand')

In [4]:
tokenizer = AutoTokenizer.from_pretrained('../0_models/bert-rand-1m-3ep-rand', use_fast=True)

trainer = Trainer(model = model,tokenizer = tokenizer)

In [33]:
datasets = load_dataset('csv', data_files={'test': '../0_data/clean/labelled_ghc/eval_random_small.csv'})

Using custom data configuration default-102372a7319b6159
Reusing dataset csv (/Users/Paul/.cache/huggingface/datasets/csv/default-102372a7319b6159/0.0.0/965b6429be0fc05f975b608ce64e1fa941cc8fb4f30629b523d2390f3c0e1a93)


In [42]:
label_list = datasets["test"].unique("label")
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)


label_to_id = {v: i for i, v in enumerate(label_list)}

def preprocess_function(examples):
        result = tokenizer(examples['clean_text'], truncation=True)
        if label_to_id is not None and "label" in examples:
            result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
        return result

datasets = datasets.map(preprocess_function, batched=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [46]:
test_dataset = datasets['test']

In [47]:
pred_results = trainer.predict(test_dataset)

In [70]:
labels = pd.DataFrame(pred_results.label_ids, columns = ['label']).reset_index()
predictions = pd.DataFrame(np.argmax(pred_results.predictions, axis=1), columns = ['prediction']).reset_index()
output = labels.merge(predictions).drop(columns = ['index'])
output

,label,prediction
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
5,0,1
6,1,1
7,0,1
8,0,1
9,0,1
